## Imports

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path('.').absolute().parent))

import numpy as np
import matplotlib.pyplot as plt
import optimizationFuncs as optim

## Derivative Approximation

In [ ]:
def objder (x, func) :
    h = 1e-8
    disps = []
    for i in range(x.shape[0]) :
        disp = np.copy(x)
        disp[i] += h
        disps.append(disp)
        
    return np.array([
        (func(d.reshape(1, -1)) - func(x.reshape(1, -1)))[0]/h
        for d in disps
    ])

## Plain Gradient Descent

In [ ]:
def grad_desc (x, func, iters=100) :
    alpha = 1e-7
    for _ in range(1000) :
        x -= alpha*objder(x, func)
    
    return x

## Adam

In [ ]:
def adam(x, grad, llim, rlim, min_iters=100, max_iters=10000,
         step_size=0.001, b1=0.9, b2=0.999, eps=10**-8):
    m = np.zeros(len(x))
    v = np.zeros(len(x))
    i = 0
    while True :
        g = grad(x)
        gt = (np.abs(g) < 1e-5).all()
        if i == max_iters :
            return x, gt, 'max_iter'
        elif i >= min_iters and gt :
            return x, gt, 'proper'
        
        m = (1 - b1) * g      + b1 * m 
        v = (1 - b2) * (g**2) + b2 * v
        mhat = m / (1 - b1**(i + 1))
        vhat = v / (1 - b2**(i + 1))
        
        xsave = x
        x = x - step_size*mhat/(np.sqrt(vhat) + eps)
        if not np.logical_and(llim <= x, x <= rlim).all() :
            return xsave, gt, 'lim_violated'
        
        i += 1

## Test with Seeds

In [ ]:
no_seeds = 100
dims = 5
llim = np.repeat (0, dims)
rlim = np.repeat (10, dims)

seeds = np.array([l + (r-l)*np.random.rand(no_seeds, dims).transpose()[ind] \
                              for ind, l, r in zip(range(0,dims), llim, rlim)]).transpose()
func = optim.rastrigin
seed_mins = np.array([
    a[0] for s in seeds
    if (a:=adam(s, lambda x : objder(x, func), llim, rlim))[2] == 'proper' 
        or a[1]
])

min_ders = np.array([
    objder(sm, func) for sm in seed_mins
])

## Clustering the Seeds

In [ ]:
clusts = [seed_mins[0]]
for smin in seed_mins[1:] :
    if not (np.abs(objder(smin, func)) < 1e-4).all() :
        continue
    
    chk = np.array([
        (np.abs(smin - sm) < 1e-4).all()
        for sm in clusts
    ])
    
    if not chk.any() :
        clusts.append(smin)

clusts = np.array(clusts)
print (len(clusts), len(seed_mins))

## Finding foreign minima

In [ ]:
i = 1
revseed = clusts[i]
x = np.random.rand(dims) - 0.5
nx = x / np.linalg.norm(x)

diff = clusts - revseed

perp_strict = np.array([
    np.inf if not (nx*d).all() or i == j else np.sum(np.square(d)*(1-np.square(nx))) 
    for j, d in enumerate(clusts - revseed)
])

perp = np.array([
    np.inf if i == j else np.sum(np.square(d)*(1-np.square(nx))) 
    for j, d in enumerate(clusts - revseed)
])

print (revseed)
if (perp_strict == np.inf).all() :
    print (clusts[np.argmin(perp)])
else :
    print (clusts[np.argmin(perp_strict)])